In [39]:
import os

import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from colorthief import ColorThief
from PIL import Image, ImageFont, ImageDraw, ImageFilter

In [40]:
images_loc = "/Volumes/BCross/Documents/YouTube/Images/"

image_save_loc = "/Volumes/BCross/Documents/YouTube/Formatted Images/"
short_image_save_loc = "/Volumes/BCross/Documents/YouTube/Short Formatted Images/"

thumbnail_save_loc = "/Volumes/BCross/Documents/YouTube/Thumbnails/"
short_thumbnail_save_loc = "/Volumes/BCross/Documents/YouTube/Short Thumbnails/"

In [41]:
historical_figures_list = pd.read_excel(r'../Historical Figures List.xlsx')
short_historical_figures_list = pd.read_excel(r'../Short Historical Figures List.xlsx')

In [42]:
# We grab any that still need images
need_images = historical_figures_list[(historical_figures_list.Script_Created == "Yes") &
                                      (historical_figures_list.AI_Voice_Generated == "Yes") &
                                      (historical_figures_list.Images_Obtained == "No")]
need_images.head(5)

,Figure_ID,Name,Description,Script_Created,AI_Voice_Generated,Images_Obtained,Youtube_Video_Created,Youtube_Video_Posted,Youtube_URL,Word_Count,Model_Used,Wikipedia
192,193,Qutb-ud-din Aybak,Slave-turned-sultan who founded the Delhi Sult...,Yes,Yes,No,No,No,NaN,763.0,gpt-4-1106-preview,unavailable
193,194,Wang Zhenyi,An 18th-century Chinese astronomer and mathema...,Yes,Yes,No,No,No,NaN,989.0,gpt-4-1106-preview,unavailable
194,195,Claudette Colvin,An American nurse aid who was a pioneer of the...,Yes,Yes,No,No,No,NaN,725.0,gpt-4-1106-preview,unavailable
195,196,Maria Mutola,"Mozambican middle-distance runner, Olympic gol...",Yes,Yes,No,No,No,NaN,969.0,gpt-4-1106-preview,unavailable
196,197,Harvey Milk,One of the first openly gay elected officials ...,Yes,Yes,No,No,No,NaN,1121.0,gpt-4-1106-preview,https://en.wikipedia.org/wiki/Harvey_Milk


In [43]:
# We grab any that still need images
short_need_images = short_historical_figures_list[(short_historical_figures_list.Script_Created == "Yes") &
                                      (short_historical_figures_list.AI_Voice_Generated == "Yes") &
                                      (short_historical_figures_list.Images_Obtained == "No")]
short_need_images.head(5)

,Figure_ID,Name,Description,Script_Created,AI_Voice_Generated,Images_Obtained,Youtube_Video_Created,Youtube_Video_Posted,Youtube_URL,Short_Youtube_URL,Word_Count,Model_Used,Wikipedia


In [43]:
def convert_images_from_folder(figure, yt_short=False, update_excel=True):

    if yt_short == False:
        thumbnail_loc = thumbnail_save_loc
        save_loc = image_save_loc
        image_height = 720
        image_width = 1280
    else:
        thumbnail_loc = short_thumbnail_save_loc
        save_loc = short_image_save_loc
        image_height = 1280
        image_width = 720

    image_path = images_loc + figure
    save_location = save_loc + figure
    thumbnail_save_location = thumbnail_loc + figure

    # Create directories if they do not exist
    os.makedirs(save_location, exist_ok=True)
    os.makedirs(thumbnail_save_location, exist_ok=True)
    
    quality_val = 90
    images = os.listdir(image_path)
    
    # Remove '.DS_Store' and '.avif' files
    images = [i for i in images if i != '.DS_Store' and not i.endswith('.avif')]
    
    images_kept = 1
    thumbnail_options = 1
    
    for image in images:
        
        image_location = image_path + "/" + image
        temp_save_location = save_loc + figure + "/test.png"
   
        # Load as image object
        selected_image = Image.open(image_location)
        selected_image.save(temp_save_location)

        # This plots the image so we can see it
        %matplotlib inline
        img = mpimg.imread(temp_save_location)
        imgplot = plt.imshow(img)
        plt.show()
        
        # Text to whether i want to save the image as a thumbnail option
        thumbnail = input("Use as thumbnail? Y or N: ")
                
        if thumbnail == "Y" or thumbnail == "y":
            selected_image.save(thumbnail_save_location + "/thumbnail" + str(thumbnail_options) + ".png", 
                                quality=quality_val)
            thumbnail_options += 1
                
        # We want all images to be the same size. If they aren't resize them.
        h = selected_image.height
        w = selected_image.width
                
        if h > image_height:
            h = image_height
            selected_image = selected_image.resize((w, h), Image.LANCZOS)
        if w > image_width:
            w = image_width
            selected_image = selected_image.resize((w, h), Image.LANCZOS)

        start_pos_w = (image_width / 2) - w/2
        start_pos_h = (image_height / 2) - h/2
 
        # Want to paste the resized image onto dominant colour background
        color_thief = ColorThief(temp_save_location)
        # get the dominant color
        dominant_color = color_thief.get_color(quality=1)
        # Background
        bk = Image.new('RGB', (image_width, image_height), color = dominant_color)
        # paste background image on to border image
        bk.paste(selected_image, (int(start_pos_w), int(start_pos_h)))
                
        bk.save(save_location + "/" + str(images_kept) + '.png',
                quality=quality_val)
                
        if yt_short == False:

            if update_excel:
                # Change the image obtained to yes for the specified figure and save
                historical_figures_list.loc[(historical_figures_list.Name == figure), "Images_Obtained"] = "Yes"
                historical_figures_list.to_excel(r'../Historical Figures List.xlsx', index=False)

        else:

            if update_excel:
                # Change the image obtained to yes for the specified figure and save
                short_historical_figures_list.loc[(short_historical_figures_list.Name == figure), "Images_Obtained"] = "Yes"
                short_historical_figures_list.to_excel(r'../Short Historical Figures List.xlsx', index=False)
        
        # Remove the test image
        os.remove(temp_save_location)
        
        # os.remove(image_location)
        
        images_kept += 1
        
    print("All Done Laa!")

In [44]:
# convert_images_from_folder('Ivan Lendl', yt_short = True)

In [45]:
# convert_images_from_folder('Ivan Lendl')